# Build Sample Logistic Model to Get Baseline Check of Raw Data

## This script will import CSV files directly
sql_connection.ipynb will test the connection to the postgres database before both are combined into a .py file

In [11]:
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
cleaned_data_url = '../Resources/2018MBB_StatsAndDraft_Cleaned.csv'

raw_df = pd.read_csv(cleaned_data_url)

In [3]:
raw_df.head()

,Player,Team,GP,MPG,FGM,FGA,FG%,3PM,3PA,3P%,...,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Pk,Player1
0,Deandre Ayton,U of A,35,33.5,7.9,12.9,0.612,0.3,1.0,0.343,...,2.3,3.4,8.2,11.6,1.6,0.6,1.9,20.1,1.0,Deandre Ayton
1,Marvin Bagley III,DUKE,33,33.9,8.2,13.3,0.614,0.7,1.8,0.397,...,1.8,4.0,7.1,11.1,1.5,0.8,0.9,21.0,2.0,Marvin Bagley III
2,Trae Young,OU,32,35.4,8.2,19.3,0.422,3.7,10.2,0.360,...,1.8,0.4,3.5,3.9,8.7,1.7,0.2,27.4,5.0,Trae Young
3,Mo Bamba,UT,30,30.2,4.9,9.0,0.541,0.5,1.7,0.275,...,2.5,3.2,7.3,10.5,0.5,0.8,3.7,12.9,6.0,Mo Bamba
4,Collin Sexton,UA,33,29.9,5.9,13.3,0.447,1.3,4.0,0.336,...,2.5,1.0,2.7,3.8,3.6,0.8,0.1,19.2,8.0,Collin Sexton


## Preprocessing
Limited preprocessing done at this time
* No scaling or encoding  
* All numerical data used as-is  
* All categorical & text data dropped

In [4]:
numerical_cols = raw_df.dtypes[raw_df.dtypes != 'object'].index.tolist()

In [5]:
numerical_df = raw_df[numerical_cols]
numerical_df.index = raw_df['Player']
numerical_df.head()

,GP,MPG,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,TOV,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Pk
Player,,,,,,,,,,,,,,,,,,,,,
Deandre Ayton,35,33.5,7.9,12.9,0.612,0.3,1.0,0.343,4.0,5.5,...,2.0,2.3,3.4,8.2,11.6,1.6,0.6,1.9,20.1,1.0
Marvin Bagley III,33,33.9,8.2,13.3,0.614,0.7,1.8,0.397,4.0,6.3,...,2.3,1.8,4.0,7.1,11.1,1.5,0.8,0.9,21.0,2.0
Trae Young,32,35.4,8.2,19.3,0.422,3.7,10.2,0.360,7.4,8.6,...,5.2,1.8,0.4,3.5,3.9,8.7,1.7,0.2,27.4,5.0
Mo Bamba,30,30.2,4.9,9.0,0.541,0.5,1.7,0.275,2.7,4.0,...,1.5,2.5,3.2,7.3,10.5,0.5,0.8,3.7,12.9,6.0
Collin Sexton,33,29.9,5.9,13.3,0.447,1.3,4.0,0.336,5.9,7.6,...,2.8,2.5,1.0,2.7,3.8,3.6,0.8,0.1,19.2,8.0


In [28]:
y.value_counts()

0    1963
1      22
2      15
Name: Pk, dtype: int64

In [6]:
X = numerical_df.drop(columns=['Pk'])
y = numerical_df['Pk'].apply(lambda x: 1 if x <= 30 else 2 if x <= 60 else 0)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

## Set up sample Logistic Regression Model

In [8]:
classifier = LogisticRegression(solver='lbfgs',random_state=1, max_iter=1000)
classifier.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [9]:
predictions = classifier.predict(X_test)

In [10]:
acc_score = accuracy_score(y_test,predictions)
acc_score

0.986

In [14]:
matrix = confusion_matrix(y_test,predictions)
print(matrix)

[[493   1   0]
 [  5   0   0]
 [  1   0   0]]


In [15]:
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

In [22]:
results[results['Prediction'] != 0]

,Prediction,Actual
93,1,0


In [24]:
report = classification_report(y_test,predictions,zero_division=True)

In [25]:
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       494
           1       0.00      0.00      0.00         5
           2       1.00      0.00      0.00         1

    accuracy                           0.99       500
   macro avg       0.66      0.33      0.33       500
weighted avg       0.98      0.99      0.98       500



## Next steps:
1. Test other models, starting with SVM (binary classifier)
2. PCA to find most important features